# Classification Model

In [10]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.4.0-dev20200902


In [11]:
import tensorboard
tensorboard.__version__

'2.5.0a20201219'

In [13]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [1]:
pip install tflite-model-maker

  Using cached https://files.pythonhosted.org/packages/dc/9e/03d8ada09b393447c0edf36ff74e72682b2e8c1f5f46e9bf2ce73297f22f/tf_nightly-2.4.0.dev20200902-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/6d/45/1b79a69483dd2632e21b56d4511b7d6efdf774656a915772ccab1d874914/tf_estimator_nightly-2.4.0.dev2020090201-py2.py3-none-any.whl
     |████████████████████████████████| 2.9MB 4.2MB/s 
ERROR: tf-nightly 2.4.0.dev20200902 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
  Found existing installation: tf-estimator-nightly 2.5.0.dev2020121901
    Uninstalling tf-estimator-nightly-2.5.0.dev2020121901:
      Successfully uninstalled tf-estimator-nightly-2.5.0.dev2020121901
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: tf-nightly 2.5.0.dev20201219
    Uninstalling tf-nightly-2.5.0.dev20201219:
      Successfully unins

In [5]:
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker import TextClassifierDataLoader

In [6]:
spec = model_spec.get('mobilebert_classifier')

In [7]:
train_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join('/content/yotube_emenem_cleaned_train.csv')),
      text_column='CONTENT',
      label_column='CLASS',
      model_spec=spec)
test_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join('/content/yotube_emenem_cleaned_test.csv')),
      text_column='CONTENT',
      label_column='CLASS',
      model_spec=spec)

In [15]:
model = text_classifier.create(train_data, model_spec=spec,epochs=6  )

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/6
7/7 [==============================] - 204s 20s/step - loss: 4.8256 - test_accuracy: 0.5299
Epoch 2/6
7/7 [==============================] - 132s 19s/step - loss: 1.3581 - test_accuracy: 0.5426
Epoch 3/6
7/7 [==============================] - 131s 19s/step - loss: 0.5651 - test_accuracy: 0.7347
Epoch 4/6
7/7 [==============================] - 130s 19s/step - loss: 0.3895 - test_accuracy: 0.8130
Epoch 5/6
7/7 [==============================] - 132s 19s/step - loss: 0.3392 - test_accuracy: 0.8664
Epoch 6/6
7/7 [==============================] - 131s 19s/step - loss: 0.1993 - test_accuracy: 0.9254


In [16]:
loss, acc = model.evaluate(test_data)

4/4 [==============================] - 19s 3s/step - loss: 0.1666 - test_accuracy: 0.9375


In [22]:
#Since MobileBERT is too big for on-device applications 
#use dynamic range quantization on the model to compress it by almost 4x with minimal performance degradation.
config = configs.QuantizationConfig.create_dynamic_range_quantization(optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_LATENCY])
config._experimental_new_quantizer = True

In [23]:
model.export(export_dir='mobilebert/', quantization_config=config)

INFO:tensorflow:Assets written to: /tmp/tmpil6dm4n0/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpil6dm4n0/saved_model/assets


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Saved vocabulary in /tmp/tmpozc9mbt3/vocab.txt.


INFO:tensorflow:Saved vocabulary in /tmp/tmpozc9mbt3/vocab.txt.


INFO:tensorflow:Saving labels in /tmp/tmpozc9mbt3/labels.txt.


INFO:tensorflow:Saving labels in /tmp/tmpozc9mbt3/labels.txt.


Finished populating metadata and associated file to the model:
mobilebert/model.tflite
The metadata json file has been saved to:
mobilebert/model.json
The associated file that has been been packed to the model is:
['labels.txt', 'vocab.txt']


In [24]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(1, 128)]           0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(1, 128)]           0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(1, 128)]           0                                            
__________________________________________________________________________________________________
hub_keras_layer_v1v2_1 (HubKera (None, 512)          24581888    input_word_ids[0][0]             
                                                                 input_mask[0][0]           